In [46]:
import mlflow.pyfunc
from datetime import timedelta
import pandas as pd
import dagshub
import yfinance as yf

In [47]:
!pip install mlflow==2.16.1
!pip install cloudpickle==3.0.0
!pip install numpy==2.1.1
!pip install pandas==2.2.2
!pip install psutil==6.0.0


In [48]:
# Definir la configuración del modelo y el alias del modelo optimizado
dagshub.init(url="https://dagshub.com/PacoTinoco/Proyecto_Final_CDD", mlflow=True)
MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

model_name = "amazon-stock-model-randomforest-prefect"
alias = "champion"
model_uri = f"models:/{model_name}@{alias}"

# Cargar el modelo entrenado de MLflow
model = mlflow.pyfunc.load_model(model_uri)


Initialized MLflow to track repo "PacoTinoco/Proyecto_Final_CDD"

Repository PacoTinoco/Proyecto_Final_CDD initialized!

In [49]:
# Solicitar datos al usuario
capital_inicial = float(input("Ingrese el capital inicial a invertir en USD: "))
dias_inversion = int(input("Ingrese el número de días de inversión: "))

In [50]:
def obtener_datos_limpios(ticker, periodo="1mo"):
    stock = yf.Ticker(ticker)
    df = stock.history(period=periodo)
    columnas_limpias = {
        'Open': 'Open',
        'High': 'High',
        'Low': 'Low',
        'Close': 'Close',
        'Volume': 'Volume'
    }
    df = df.rename(columns=columnas_limpias)
    df = df.reset_index()
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    return df
data = obtener_datos_limpios('AMZN', '1mo')
data = data.drop(['Dividends', 'Stock Splits'], axis=1)
data



,Date,Open,High,Low,Close,Volume
0,2024-10-21,188.050003,189.460007,186.399994,189.070007,24639400
1,2024-10-22,188.350006,191.520004,186.979996,189.699997,29650600
2,2024-10-23,188.850006,189.160004,183.690002,184.710007,31937100
3,2024-10-24,185.250000,187.110001,183.860001,186.380005,21647400
4,2024-10-25,187.850006,190.449997,187.529999,187.830002,29362100
5,2024-10-28,189.570007,190.210007,188.210007,188.389999,27930800
6,2024-10-29,188.580002,191.460007,187.820007,190.830002,35690200
7,2024-10-30,194.699997,195.610001,192.419998,192.729996,37707600
8,2024-10-31,190.509995,190.600006,185.229996,186.399994,75146800
9,2024-11-01,199.000000,200.500000,197.020004,197.929993,99687800


In [51]:
data.to_csv('Data_amazon_definitiva.csv', index=False)


In [52]:
input_data = data.iloc[-1:]
input_data

,Date,Open,High,Low,Close,Volume
21,2024-11-19,199.330002,205.300003,198.779999,204.610001,30916900


In [53]:
data.columns



Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [54]:
# Iterar para simular la predicción de cada día adicional
precio_inicial = input_data[('Open')]


In [55]:
cantidad_acciones = capital_inicial / precio_inicial

In [56]:
precio_estimado = model.predict(input_data)

In [57]:
precio_inicial

21    199.330002
Name: Open, dtype: float64

In [58]:
precio_siguiente = precio_inicial
for _ in range(dias_inversion):
    # Calculamos la proporción del precio estimado respecto al precio inicial
    proporcion = precio_estimado / precio_inicial
    # Actualizamos el precio inicial para el siguiente día
    precio_siguiente = precio_siguiente * proporcion
    
precio_final = precio_siguiente
precio_final

21    91.557969
Name: Open, dtype: float64

In [59]:
cantidad_acciones = capital_inicial / precio_inicial
capital_final = cantidad_acciones * precio_final
capital_final

21    4593.285905
Name: Open, dtype: float64

In [60]:
capital_inicial, capital_final

(10000.0,
 21    4593.285905
 Name: Open, dtype: float64)

In [62]:
rendimiento = capital_final/capital_inicial
rendimiento

21    0.459329
Name: Open, dtype: float64